# Project Part 2

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/sgeinitz/CS39AA-project/blob/main/project_part1.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sgeinitz/CS39AA-project/blob/main/project_part1.ipynb)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

url = 'https://raw.githubusercontent.com/PGLavergne/NYTCrosswordPredicter/main/nytcrosswords.csv'
dataSet = pd.read_csv(url, encoding='latin-1')
dataSet_partial = pd.read_csv(url, encoding='latin-1', encoding='latin-1')
dataSet

In [ ]:
#this code calculates the clue/answer lengths in terms of words/chars, respectively. 
dataSet_partial["Clue_Length_Words"] = dataSet_partial['Clue'].apply(lambda x: len(x.split()))
dataSet_partial['Word_Length_Chars'] = dataSet_partial['Word'].apply(len)

plt.figure(figsize=(12, 5))

plt.subplot(1,2,1)
plt.hist(dataSet_partial['Clue_Length_Words'], bins=20, color='green')
plt.title('Hist of Clue Length (Words)')
plt.xlable('Num of Words')
plt.ylabel('Frequency')

plt.subplot(1,2,2)
plt.hist(dataSet_partial['Word_Length_Chars'], bins=20, color='skyblue')
plt.title('Hist of Answer Length (char)')
plt.xlabel("Num of Chars")
plt.ylabel('Frequency')

plt.titght_layout()
plt.show()